In [1]:
import pyaudio
import wave
import matplotlib.pyplot as plt
import numpy as np

#man pacat for unix interface
#libpulse : ldconfig -p | grep pulse

from scipy.fft import rfft, irfft
import math
from scipy.stats import norm as normal_dist

In [2]:
CHUNK = 4096
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 15
WAVE_OUTPUT_FILENAME = "output.wav"

In [3]:
def harmonic_similarity_matrix(window, rate, sigma = 1/36, num_harmonics = 1):
    len_fft = window//2 + 1
    notes = [27.5 * 2**(i/12) for i in range(120)]
    
    mat = np.zeros((len_fft, len(notes)))
    # each value in mat is the probability that a fft frequency belongs to a note or it's harmonic series
    temp = np.zeros((len_fft, len(notes)))
    
#     log_freq = (math.log(55,2) + np.arange(0,8,(1/12)))
#     log_fft = np.log2(np)
    
    for h in range(1,1 + num_harmonics):
        for i in range(1, len_fft):
#             print(i, end="\r")
            fft_n = i * (rate/window)
            for j in range(len(notes)):
                note = notes[j]
                factor = h
                freq = factor * note
                v = abs(math.log(freq,2) - math.log(fft_n,2))
                temp[i][j] = (v/sigma)
        mat += 2 * (1- normal_dist().cdf(temp))
    mat[0][:] = 0.0
#     for i in range(len_fft):
#         s = sum(abs(mat[i])) 
#         if s > 0.0:
#             mat[i] /= s
            
    return mat
                # 

In [4]:
dt = np.dtype(np.int16)
dt = dt.newbyteorder('<')

hmat = harmonic_similarity_matrix(CHUNK, RATE)

# samples = np.empty((0), dtype=dt)
# samples
# fft_s = np.zeros((CHUNK//2, CHUNK//2))
# fft_c = np.zeros((CHUNK//2, CHUNK//2))
# FxH = FHx

# 2048 vector -> 120 vector (multiply by 2048x120)

names = ["A BbB C DbD EbE F GbG Ab"[(i%12)*2:(i%12)*2+2].strip() + str(i//12) for i in range(120)]

In [ ]:
fig, ax = plt.subplots(figsize=(10,450))
im = ax.imshow(hmat)

In [ ]:
p = pyaudio.PyAudio()


# input("Press any key to start recording")

print("* recording")

frames = []

maxes = []

stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

last = ""

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
    sample = np.frombuffer(data, dtype=dt)
    l = np.matmul( np.abs(rfft(sample)) , hmat )
    maxes.append(l)
    note = names[np.argmax(l)]
    if note != last:
        print(note)
        last = note
    

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

In [5]:
wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

NameError: name 'p' is not defined

In [ ]:
idx = 0
for f in frames:
    plt.figure(figsize=(30,10))
    
    raw = np.frombuffer(f, dtype=dt)
    
#     avg = (0.25*raw[0::4] + 0.25*raw[1::4] + 0.25*raw[2::4] + 0.25*raw[3::4])
#     plt.plot(avg)
    print(idx)
    plt.plot(raw)
    plt.show()
    idx += 1

In [ ]:
idx = 0
for m in maxes:
    print(idx, np.argmax(m))
    plt.plot(m)
    plt.show()
    idx += 1